#Paquetes necesarios

In [ ]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt

TAREA: Captura una o varias imágenes con monedas no solapadas y algún objeto que no lo sea. Filtra los contornos que no se correpondan con monedas, y muestra el número total de monedas presentes en la imagen.

In [ ]:
umbral = 130

my_img = cv2.imread('IMG_0163.jpg') 
my_img = cv2.cvtColor(my_img, cv2.COLOR_BGR2RGB)

my_img_gris = cv2.cvtColor(my_img, cv2.COLOR_BGR2GRAY)

res,my_img_th1 = cv2.threshold(my_img_gris,umbral,255,cv2.THRESH_BINARY_INV)


contornos_externos, hierarchy_externos = cv2.findContours(my_img_th1, 
    cv2.RETR_EXTERNAL , 
    cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(my_img, contornos_externos, -1, (0,255,0), 3)


count = 0
img_cont = np.zeros(my_img.shape)
for c in contornos_externos:
    area = cv2.contourArea(c)

    if area > 500:
        count += 1

        cv2.drawContours(img_cont, [c], -1, (0,0,255), -1)

        
print(f'La cantidad de monedas son: {count}')

plt.imshow(my_img) 
plt.show()
plt.imshow(img_cont) 
plt.show()

TAREA: Captura una o varias imágenes con monedas no solapadas, y otras con monedas solapadas. Identificada una moneda de un euro en la imagen, por ejemplo con un clic de ratón, calcular la cantidad de dinero presente en la imagen. ¿Qué problemas han observado?




El problema principal surgido es conseguir que cada moneda se cuente, pero que cuenta una sola vez.

Este problema es extarsobado por las monedas solapadas, las cuales obligan a reducir la distancia minima entre circulos.

Tambien existe el problema en las monedas solapadas de que circulo selecionar como la moneda de un euro.

In [ ]:
def calcular_cantidad_de_dinero_presente(img):
    cv2.destroyAllWindows()
    # Using Hugh identify all coins.
    gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #Suaviza altas frecuencias
    pimg = cv2.medianBlur(gris, 7)
    
    circ = cv2.HoughCircles(
        pimg,  # imagen 
        cv2.HOUGH_GRADIENT,  # tipo de detección
        1,
        400,  # distancia mínima entre círculos
        param1=170, # Valor del gradiente
        param2=40, #Umbral acumulador
        minRadius=300,  # radio mínimo
        maxRadius=1000,  # radio máximo
    )

    

    # Have user click on the one euro coin

    mouse_event_flag = False
    coin_value = 0
    coins = [
        (0.699354838709677, 0.01),
        (0.806451612903226, 0.02),
        (0.849462365591398, 0.1),
        (0.913978494623656, 0.05),
        (0.956989247311828, 0.2),
        (1, 1),
        (1.043010752688172, 0.5),
        (1.10752688172043, 2)
    ]
    def mouse_event(event, x, y, flags, params):
        global mouse_event_flag
        global circ
        global coin_value
        # Based on the radius of this identify all other coins and calculate total amount of cash
        
        

        if event==cv2.EVENT_LBUTTONDOWN:
            # Find coin which was clicked.
            coin_value = 0
            one_euro_radius = -1
            print('Starting circle processing.')
            for det in circ[0]:
                x_coor, y_coor, det_radio = det
                print(f'x_co {x_coor} y_cor {y_coor} x {x} y {y} rad {det_radio}')
                if (x_coor - x)**2 + (y_coor - y)**2 <= det_radio**2:
                    one_euro_radius = det_radio
                    break

            for det in circ[0]:
                x_coor, y_coor, det_radio = det
                
                ratio = det_radio / one_euro_radius

                diff = abs(ratio - coins[0][0])
                for index, coin in enumerate(coins):
                    inner_diff = abs(ratio - coin[0])

                    if inner_diff > diff:
                        print(f'index : {index} ratio: {ratio} det_radio: {det_radio} radius: {one_euro_radius}')
                        coin_value += coins[index - 1][1]
                        break
                    else:
                        diff = inner_diff


            # Set the radius of this coin as base radius.
            # Determine value of each coin based on the radius of this one coin and add to coin_value
            mouse_event_flag = True
            print(coin_value)

    img_cont = np.zeros(img.shape)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for det in circ[0]:
        x_coor, y_coor, det_radio = det
        cv2.circle(img_rgb,(int(x_coor), int(y_coor)),
            int(det_radio),(0, 255, 0), 2)
        cv2.circle(img_cont,(int(x_coor), int(y_coor)),
            int(det_radio),(255, 255, 255), -1)

    cv2.imshow('Elije una moneda de un euro', img_rgb)
    cv2.setMouseCallback('Elije una moneda de un euro', mouse_event)

    while (cv2.waitKey(20) != 27):
        pass

    return coin_value
    


img_solapada = cv2.imread('IMG_0165.jpg') 
img_no_solapada = cv2.imread('IMG_0163.jpg') 

print(calcular_cantidad_de_dinero_presente(img_solapada))
print(calcular_cantidad_de_dinero_presente(img_no_solapada))

cv2.destroyAllWindows()

Clasificación de microplásticos

In [ ]:
#Cargamos tres subimágenes de cada uno de los tres tipos considerados (el alquitrán efectivamenet no es microplástico)
imgF = cv2.imread('FRA.png') 
imgP = cv2.imread('PEL.png') 
imgT = cv2.imread('TAR.png') 

#Mostramos
plt.subplot(131)
plt.axis("off")
plt.imshow(imgF) 
plt.title('Fragmentos')
plt.subplot(132)
plt.axis("off")
plt.imshow(imgP) 
plt.title('Pellet')
plt.subplot(133)
plt.axis("off")
plt.imshow(imgT) 
plt.title('Alquitrán')
plt.show()

TAREA: Estas tres imágenes han sido extraidas de las imágenes de mayor tamaño contenidas en la carpeta. Determina patrones geométricos para cada una de las tres clases y evalúa los aciertos y fallos con las imágenes completas la matriz de confusión. Para cada clase, determina el número de muestras que se clasifican correctamente de dicha clase, y el número de muestras que se clasifica incorrectamente por cada una de las otras dos clases.

El el trabajo [SMACC: A System for Microplastics Automatic Counting and Classification](https://doi.org/10.1109/ACCESS.2020.2970498), las características geométricas utilizadas en dicho trabajo fueron:

- Área en píxeles
- Perímetro en píxeles
- Compacidad (relación del cuadrado del perímetro con el área)
- Relación del área con la del contenedor
- Relación del ancho y el alto del contenedor
- Relación entre los ejes de la elipse ajustada
- Definido el centroide, relación entre las distancias menor y mayor al contorno

Si no se quedan satisfechos con la segmentación obtenida, es el mundo real, también en el README comento técnicas recientes de segmentación, que podrían despertar su curiosidad.

A ojo humano se aprecia que los fragmentos suelen tener un colo claro. Los pellets varían en color pero tienen una forma geométrica mas redondo y el alquitrán se caracteriza por su color oscuro.

Las métricas se utilizan para mostrar la bondad de un clasificador, la siguiente celda obtiene varias para un conjunto de datos imaginario (y con etiquetas aleatorias). Si bien las trataremos con más detalle en teoría, muestro un repertorio de ellas, dando más peso a la matriz de confusión.

¿Qué es una matriz de confusión?
Se utiliza para mostrar el comportamiento de un clasificador par alas distintas clases consideradas, se relacionan las etiquetas de las muestras anotadas frente a las predichas por el clasificador. Se busca una matriz diagonal, pero la perfección es infrecuente

In [ ]:

import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)


# Numero de muestras
n = 100  
nclases = 4

# A falta de clasificador y conjunto de datos, creamos anotaciones y predicciones de forma aleatoria
# Vector aleatorio con etiquetas anotadas
y = [random.randint(0, nclases - 1) for _ in range(n)]

# Vector aleatorio con etiquetas predichas por un supuesto clasificador
y_pred = [random.randint(0, nclases - 1) for _ in range(n)]


accuracy = accuracy_score(y, y_pred)
#Para más de una clase se define la forma de promediar
precision = precision_score(y, y_pred,average='weighted')
recall = recall_score(y, y_pred,average='weighted')
f1score = f1_score(y, y_pred,average='weighted')

print(f"Accuracy (TP/(n))= {accuracy}")
print(f"Precision (TP/(TP+FP)) = {precision}")
print(f"Recall (TP/(TP+FN)) = {recall}")
print(f"F1 Score (2*(precision*recall)/(precision+recall)) = {f1score}")


conf_matrix = confusion_matrix(y, y_pred)
plt.figure(figsize=(8,8))
sns.set(font_scale = 1.75)#tamaños tipografía
sns.set(font_scale = 3.0)

ax = sns.heatmap(
        conf_matrix, # confusion matrix 2D array 
        annot=True, # Muestra números en las celdas
        fmt='d', # valores enteros
        cbar=False, # sin barra de colores
        cmap='flag', # mapa de colores
        #vmax=175 # contraste de color
    )

#Etiquetas matriz de confusión
label_font = {'size':'25'}
ax.set_xlabel("Predicha", labelpad=-0.75, fontdict=label_font)
ax.set_ylabel("Anotada", labelpad=20, fontdict=label_font)